## Thử trên YOLO nhó
### Đây là trường hợp mess detection only thôi, không có clean

In [1]:
from ultralytics import YOLO

In [2]:
model = YOLO("weights/yolo11n.pt")

In [ ]:
# Data từ link 
# https://universe.roboflow.com/loc-60-9pvlt/unclean-room/browse?queryText=&pageSize=50&startingIndex=0&browseQuery=true

results = model.train(data="./data/data.yaml", 
                      epochs=100, 
                      amp=False)


In [4]:
model.save("trained_models/mess_detector_best.pt")

In [ ]:
from pathlib import Path
import ultralytics
from ultralytics import YOLO
from PIL import Image, ImageDraw
from tqdm import tqdm
import os

list_data_test = list(Path("./data/test/images").iterdir())
labels_data_test = Path("./data/test/labels")
labels_data_test.mkdir(exist_ok=True)

answer_path = Path("./answer.txt")
answer_path.touch(exist_ok=True)

num = 0
for img_path in list_data_test:
    result = model([img_path], conf=0.4)[0]
    predictions = model(img_path)
    for r in predictions:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        im.save(("./prediction/test" + str(num) + ".png"), "PNG") 
        num += 1
    boxes = result.boxes
    img = Image.open(img_path)
    img_draw = ImageDraw.Draw(img)
    width, height = img.size

    with open(answer_path, "a") as file:
        for bbox in boxes:
            x, y, w, h = map(float, bbox.xywh[0].tolist())
            x /= width
            y /= height
            w /= width
            h /= height
            print(x, y, w, h)
            file.write(f"{img_path.stem} 0 {x} {y} {w} {h}\n")

### Test với tập data ngoài để xem thử làm ăn như nào

In [ ]:
model = YOLO("./trained_models/mess_detector_best.pt") 

from pathlib import Path
import ultralytics
from ultralytics import YOLO
from PIL import Image, ImageDraw
from tqdm import tqdm
import os

# Thay đổi đường dẫn folder ảnh muốn test
test_folder = Path("./test_other")  # Thay bằng đường dẫn thực tế
list_data_test = list(test_folder.glob("*.jpg")) + list(test_folder.glob("*.png")) + list(test_folder.glob("*.jpeg"))

# Tạo folder để lưu kết quả
prediction_folder = Path("./test_other_prediction")
prediction_folder.mkdir(exist_ok=True)

answer_path = Path("./test_other_results.txt")
# Xóa file cũ nếu có
if answer_path.exists():
    answer_path.unlink()
answer_path.touch()

print(f"Found {len(list_data_test)} images to process")

num = 0
for img_path in tqdm(list_data_test, desc="Processing images"):
    try:
        # Detect objects
        results = model([img_path], conf=0.4)
        result = results[0]
        
        # Vẽ bounding boxes và lưu ảnh
        if len(result.boxes) > 0:  # Chỉ lưu ảnh có detection
            im_array = result.plot()  # plot a BGR numpy array of predictions
            im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
            output_path = prediction_folder / f"{img_path.stem}_detection.png"
            im.save(output_path)
            
            # Lưu thông tin detection
            boxes = result.boxes
            img = Image.open(img_path)
            width, height = img.size
            
            with open(answer_path, "a") as file:
                for bbox in boxes:
                    x, y, w, h = map(float, bbox.xywh[0].tolist())
                    conf = float(bbox.conf[0])  # Confidence score
                    x_norm = x / width
                    y_norm = y / height
                    w_norm = w / width
                    h_norm = h / height
                    
                    # Ghi kết quả với confidence score
                    file.write(f"{img_path.stem} 0 {x_norm:.6f} {y_norm:.6f} {w_norm:.6f} {h_norm:.6f} {conf:.4f}\n")
            
            print(f"✓ {img_path.name}: Found {len(boxes)} objects")
        else:
            print(f"✗ {img_path.name}: No objects detected")
        
        num += 1
        
    except Exception as e:
        print(f"Error processing {img_path.name}: {e}")

print(f"\nCompleted! Processed {num} images")
print(f"Results saved to: {answer_path}")
print(f"Images with detections saved to: {prediction_folder}")

### Train thêm trên nền data của Kiệt

In [2]:
model = YOLO("trained_models/mess_detector_best.pt")

In [3]:
results = model.train(data="./data_kiet/data.yaml", 
                      epochs=100, 
                      amp=False)

# Save model
model.save("./trained_models/mess_detector_best_v2.pt")

Ultralytics 8.3.157  Python-3.10.11 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data_kiet/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=trained_models/mess_detector_best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train18, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, per

train: Scanning D:\Coding\AI_MachineLearning\cleaning-service-recommendation\thong\data_kiet\train\labels.cache... 576 images, 288 backgrounds, 0 corrupt: 100%|██████████| 576/576 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 513.5258.4 MB/s, size: 50.7 KB)


val: Scanning D:\Coding\AI_MachineLearning\cleaning-service-recommendation\thong\data_kiet\valid\labels.cache... 20 images, 10 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to c:\Users\mytho\runs\detect\train18\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to c:\Users\mytho\runs\detect\train18
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      4.56G     0.3176     0.2856     0.9135         17        640: 100%|██████████| 36/36 [00:08<00:00,  4.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.33it/s]

                   all         20         12      0.431      0.333      0.224     0.0693



100 epochs completed in 0.258 hours.
Optimizer stripped from c:\Users\mytho\runs\detect\train18\weights\last.pt, 5.5MB
Optimizer stripped from c:\Users\mytho\runs\detect\train18\weights\best.pt, 5.5MB

Validating c:\Users\mytho\runs\detect\train18\weights\best.pt...
Ultralytics 8.3.157  Python-3.10.11 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  7.12it/s]


                   all         20         12      0.526      0.417      0.384      0.222
Speed: 0.3ms preprocess, 3.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to c:\Users\mytho\runs\detect\train18


In [6]:
# Test
from pathlib import Path
import ultralytics
from ultralytics import YOLO
from PIL import Image, ImageDraw
from tqdm import tqdm
import os

list_data_test = list(Path("./data_kiet/test/images").iterdir())
labels_data_test = Path("./data_kiet/test/labels")
labels_data_test.mkdir(exist_ok=True)

answer_path = Path("./answer.txt")
answer_path.touch(exist_ok=True)

num = 0
for img_path in list_data_test:
    result = model([img_path], conf=0.4)[0]
    predictions = model(img_path)
    for r in predictions:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        im.save(("./prediction/test" + str(num) + ".png"), "PNG") 
        num += 1
    boxes = result.boxes
    img = Image.open(img_path)
    img_draw = ImageDraw.Draw(img)
    width, height = img.size

    with open(answer_path, "a") as file:
        for bbox in boxes:
            x, y, w, h = map(float, bbox.xywh[0].tolist())
            x /= width
            y /= height
            w /= width
            h /= height
            print(x, y, w, h)
            file.write(f"{img_path.stem} 0 {x} {y} {w} {h}\n")


0: 640x640 (no detections), 28.4ms
Speed: 4.0ms preprocess, 28.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 d:\Coding\AI_MachineLearning\cleaning-service-recommendation\thong\data_kiet\test\images\0_png.rf.a1a9446a815cb3050189fdb5681d15b8.jpg: 640x640 1 mess, 28.4ms
Speed: 3.7ms preprocess, 28.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 28.1ms
Speed: 3.2ms preprocess, 28.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 d:\Coding\AI_MachineLearning\cleaning-service-recommendation\thong\data_kiet\test\images\1_png.rf.af18bd5f273caacac04eca955819bdb4.jpg: 640x640 (no detections), 28.1ms
Speed: 6.5ms preprocess, 28.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 mess, 28.6ms
Speed: 4.2ms preprocess, 28.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 d:\Coding\AI_MachineLearning\cleaning-service-recommenda